In [1]:
import pandas as pd
import sqlite3
from recipe_query import query_recipes
from dash import Dash, html, dcc, ctx
from dash.dependencies import Input, Output, State

## set up `ingredients` table

In [2]:
# Global database connection variable
recipes_db = None

def get_recipes_db():
    """Ensures a global database connection is available and returns it, setting it up if not already configured."""
    global recipes_db

    if recipes_db is not None: # if database is not empty
        return recipes_db
    else:
        # Connect to the database recipes_db.sqlite
        recipes_db = sqlite3.connect("recipes_db.sqlite", check_same_thread=False)

        # SQL command to create an `ingredients` table in the database if it does not exist
        cmd = '''
        CREATE TABLE IF NOT EXISTS ingredients (
            ingredient TEXT NOT NULL UNIQUE
        );
        '''
        cursor = recipes_db.cursor()
        cursor.execute(cmd)
        recipes_db.commit()  # saves changes
        cursor.close()  # closes cursor

    return recipes_db

## Add `recipes` table to database

In [3]:
def setup_database():
    """ Imports data from CSV to SQL table using the global connection. """
    db = get_recipes_db()

    # Load data from a CSV file into a DataFrame
    recipes = pd.read_csv("../datasets/cleaned_recipes.csv")
    # Insert data from DataFrame to the SQL table, replacing if exists
    recipes.to_sql("recipes", db, if_exists="replace", index=False)

    # Verification query to check the tables in the database
    cursor = db.cursor()
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table'")
    tables = cursor.fetchall()
    cursor.close()

    return tables

# Call setup_database function and print the tables to verify
tables = setup_database()
print(tables)

[('ingredients',), ('recipes',)]


In [4]:
def insert_ingredient(ingredient):
    """
    Inserts a new ingredient into the database.
    Args:
        ingredient (str): The name of an ingredient.
    """

    # creating a cursor to our database

    db = get_recipes_db()
    cursor = db.cursor()
    try:
        cursor.execute("INSERT INTO ingredients (ingredient) VALUES (?)", (ingredient,))
        db.commit()
        return True  # Insertion was successful
    except sqlite3.IntegrityError:
        return False  # Insertion failed due to duplicate ingredient
    finally:
        cursor.close()

def fetch_ingredients():
    """ Fetches all ingredients from the database to display them """
    db = get_recipes_db()
    cursor = db.cursor()
    cursor.execute("SELECT ingredient FROM ingredients")
    ingredients = cursor.fetchall()
    cursor.close()
    return ingredients

In [5]:
import dash
from dash import Dash, dcc, Output, Input, State, html, callback_context
import dash_bootstrap_components as dbc

#building the components
app = dash.Dash(__name__, external_stylesheets=[
    "https://fonts.googleapis.com/css2?family=Chewy&display=swap", "https://fonts.googleapis.com/css2?family=Nunito&display=swap"
]) #initialize our app

#button styling (for ingredients)
header_style= {
            'textAlign': 'center',
            'color': '#F2C078',
            'fontSize': '35px',
            'fontWeight': 'bold',
            'backgroundColor': '#7EBC89',
            'padding': '10px',
            'fontFamily': 'Chewy',
            'borderRadius': '10px 10px 0 0',
            'width': '100%',
            'margin': '0'
        }
ingredients_section= {'fontFamily': 'Nunito, sans-serif', 'color': '#FE5D26','marginBottom': '10px', 'textAlign': 'center', 'fontSize': '25px'}
button_style= {'margin': '5px', 'fontFamily': 'Chewy','padding': '10px 10px', 
               'backgroundColor': '#7EBC89', 'color': 'white', 'border': 'none', 'borderRadius': '8px', 'cursor': 'pointer' }

app.layout = html.Div([
    #title section
    html.Div([
        html.H1("No-Plan Pantry", style={
            'textAlign': 'center', 
            'color': '#7EBC89', 
            'fontFamily': 'Chewy',
            'marginBottom': '20px',
            'fontSize': '40px'
        })
    ]),

    #description section
    html.Div([
        html.P(
            "Discover a new way to minimize food waste and unlock your inner chef. This application is designed to help you create delicious, easy-to-make recipes using leftover ingredients from your fridge. Simply input the items you have on hand, and let our app suggest creative meal ideas that are both tasty and resourceful. Whether you're looking to save time, reduce waste, or explore new recipes, this tool is here to inspire your cooking adventures. Give it a try and turn your leftovers into something delicious!",
            style={
                'textAlign': 'center',
                'fontSize': '14px',
                'fontFamily': 'Nunito, sans-serif',
                'margin': '0',
                    'color': '#FE5D26'
            }
        )
    ], style={
        'backgroundColor': '#FAEDCA',
        'padding': '20px',
        'borderRadius': '10px',
        'margin': '20px auto',
        'maxWidth': '600px'
    }),

    #ingredients section
    html.Div([
        html.H2("Ingredients:", style= header_style),
        html.Div([
            #vegetables column
            html.Div([
                html.H3("Vegetables:", style= ingredients_section ),
                html.Div([
                html.Button("Onion", id="onion-btn", style= button_style),
                html.Button("Corn", id="corn-btn", style= button_style),
                html.Button("Tomatoes", id="tomatoes-btn", style= button_style),
                html.Button("Potato", id="potato-btn", style= button_style),
                html.Button("Bell Pepper", id="bell pepper-btn", style= button_style),
                html.Button("Carrots", id="carrot-btn", style= button_style),
                html.Button("Pea", id="pea-btn", style= button_style),
                html.Button("Mushroom", id="mushroom-btn", style= button_style),
                html.Button("Bean", id="bean-btn", style= button_style),
                html.Button("Basil", id="basil-btn", style= button_style),
                html.Button("Cilantro", id="cilantro-btn", style= button_style),
                html.Button("Red Pepper", id="red pepper-btn", style= button_style),
                html.Button("Spinach", id="spinach-btn", style= button_style),
                html.Button("Jalapeno", id="jalapeno-btn", style= button_style),
                html.Button("Cabbage", id="cabbage-btn", style= button_style),
                html.Button("Zuchinni", id="zucchini-btn", style= button_style),
                html.Button("Cucumber", id="cucumber-btn", style= button_style),
                html.Button("Broccoli", id="broccoli-btn", style= button_style),
                html.Button("Lettuce", id="lettuce-btn", style= button_style),
            ], style={
                'display': 'flex',
                'flexDirection': 'column',
                'justifyContent': 'space-evenly',  
                'alignItems': 'center',            
                'height': '100%',                  
                'maxHeight': 'none',             
                'overflowY': 'auto',               
                'padding': '10px',
            })
        ], style={
            'flex': '1',
            'textAlign': 'center',
            'padding': '10px'
        }),
            #proteins + grain column
            html.Div([
                #proteins section
                html.H3("Proteins:", style=ingredients_section),
                html.Div([
                html.Button("Egg", id="egg-btn", style= button_style),
                html.Button("Chicken", id="chicken-btn", style= button_style),
                html.Button("Beef", id="beef-btn", style= button_style),
                html.Button("Pork", id="pork-btn", style= button_style),
                html.Button("Bacon", id="bacon-btn", style= button_style),
                html.Button("Sausage", id="sausage-btn", style= button_style),
                html.Button("Shrimp", id="shrimp-btn", style= button_style),
                html.Button("Turkey", id="turkey-btn", style= button_style),
            ], style={
                'display': 'flex',
                'flexDirection': 'column',        
                'justifyContent': 'space-evenly',  
                'alignItems': 'center',            
                'padding': '10px'}),

                #grains section
                html.H3("Grains:", style=ingredients_section),
                html.Div([
                html.Button("Flour", id="flour-btn", style= button_style),
                html.Button("Bread", id="bread-btn", style= button_style),
                html.Button("Rice", id="rice-btn", style= button_style),
                html.Button("Oat", id="oat-btn", style= button_style),
                html.Button("Tortilla", id="tortilla-btn", style= button_style),
                html.Button("Wheat", id="wheat-btn", style= button_style),
            ], style={
                'display': 'flex',
                'flexDirection': 'column',        
                'justifyContent': 'space-evenly',
                'alignItems': 'center',            
                'padding': '10px'})
        ], style={
            'flex': '1',
            'textAlign': 'center',
            'padding': '10px',
            'borderLeft': '1px solid #A7C584',  #separator between columns
            'borderRight': '1px solid #A7C584'  #separator between columns
        }),
            
            #fruits + dairy column
            html.Div([
                #Fruits
                html.H3("Fruits:", style= ingredients_section),
                html.Div([
                html.Button("Apple", id="apple-btn", style= button_style),
                html.Button("Lime", id="lime-btn", style= button_style),
                html.Button("Orange", id="orange-btn", style= button_style),
                html.Button("Coconut", id="coconut-btn", style= button_style),
                html.Button("Strawberries", id="strawberries-btn", style= button_style),
                html.Button("Cherries", id="cherries-btn", style= button_style),
                html.Button("Avocado", id="avocado-btn", style= button_style),
                html.Button("Cranberries", id="cranberries-btn", style= button_style),
                html.Button("Banana", id="banana-btn", style= button_style),
                ],
                    style= {
                        'display': 'flex',
                        'flexDirection': 'column',          
                        'justifyContent': 'space-evenly',   
                        'alignItems': 'center',            
                        'padding': '10px',
                    }
                ),
        
                #dairy
                html.H3("Dairy:", style= ingredients_section),
                html.Div([
                html.Button("Milk", id="milk-btn", style=button_style),
                html.Button("Cheese", id="cheese-btn", style=button_style),
                html.Button("Yogurt", id="yogurt-btn", style=button_style),
                html.Button("Cream", id="cream-btn", style=button_style),
                    ],
                    style= {
                        'display': 'flex',
                        'flexDirection': 'column',
                        'justifyContent': 'space-evenly',
                        'alignItems': 'center',
                        'padding': '10px',
                    }
                )
            ], style={
                'flex': '1',
                'textAlign': 'center',
                'padding': '10px',
            }),
        ], style={
            'display': 'flex',
            'justifyContent': 'space-around',
            'alignItems': 'flex-start',
            'padding': '30px 20px',
        })
    ], style={
        'paddingTop': '10',
        'paddingBottom': '40px',
        'backgroundColor': '#FAEDCA',
        'borderRadius': '10px',
        'margin': '20px auto',
        'maxWidth': '1200px',
        'overflow': 'hidden',
    }),

        # Display Added Ingredients Section
    html.Div(id='ingredients-list', style={'marginTop': '20px', 'textAlign': 'center'}),  # Ensure this DIV is part of the layout


        # Section to display all submitted ingredients
    html.H3("Submitted Ingredients:"),
    html.Ul(id="display-ingredients", style={'marginTop': '10px', 'textAlign': 'left'}),


    #combining calories + time limit section
html.Div([
    #additional inputs section
    html.H2("Additional Inputs:", style=header_style),
    
    #calories subsection
    html.Div([
        html.H3("Calories:", style={
            'textAlign': 'center',
            'fontSize': '25px',
            'color': '#FE5D26',
            'fontFamily': 'Nunito, sans-serif',
            'marginBottom': '10px',
            'marginTop': '30px'
        }),
        html.Div([
            dcc.Input(
                id='min-calories-input',
                type='number',
                placeholder='Min Calories',
                style={'width': '180px', 'marginBottom': '10px'}
            ),
            dcc.Input(
                id='max-calories-input',
                type='number',
                placeholder='Max Calories',
                style={'width': '180px', 'marginBottom': '20px'}
            ),
        ], style={
            'display': 'flex',
            'flexDirection': 'column',   
            'padding': '20px',
            'justifyContent': 'center',
            'alignItems': 'center',
        })
    ], style={
        'paddingBottom': '10px',
        'borderBottom': '1px solid #A7C584'
    }),
    
    #time limit subsection
    html.Div([
        html.H3("Time Limit (Cook & Prep Time in Minutes):", style={
            'textAlign': 'center',
            'fontSize': '25px',
            'color': '#FE5D26',
            'fontFamily': 'Nunito, sans-serif',
            'marginBottom': '20px',
            'marginTop': '20px'
        }),
        html.Div([
            dcc.Slider(
                id='time-limit-slider',
                min=0,
                max=180,
                step=1,
                marks={i: f"{i}" for i in range(0, 181, 10)},
                value=30,
                tooltip={"placement": "bottom", "always_visible": True}
            )
        ], style={
            'padding': '20px 0',
            'fontFamily': 'Nunito, sans-serif',
            'fontSize': '20px',
            'color': '#A7C584',
            'maxWidth': '800px',
            'width': '100%',
            'margin': '0 auto',
        }),
    ], style={
        'padding': '20px',
    }),
    
    #single submit button
    html.Div([
        html.Button("Submit", id="submit-btn", style={'margin': '10px auto', 
                                                      'fontFamily': 'Chewy',
                                                      'padding': '0', 
                                                      'backgroundColor': '#7EBC89', 
                                                      'color': 'white', 
                                                      'border': 'none', 
                                                      'borderRadius': '8px', 
                                                      'cursor': 'pointer',
                                                      'width': '100px',  
                                                      'height': '40px',
                                                      'fontSize': '20px',
                                                      'textAlign': 'center',
                                                      'lineHeight': '40px'
                                                     }),
    ], style={
        'textAlign': 'center'
    })
], style={
    'paddingBottom': '20px',
    'backgroundColor': '#FAEDCA',
    'borderRadius': '10px',
    'marginBottom': '40px',
    'maxWidth': '1200px',
    'margin': '40px auto',
    'overflow': 'hidden'
}),

    #results section
    html.Div([
        html.H2("Results:", style=header_style),
        html.Div(id='results-content', style={
        'padding': '20px',
        'backgroundColor': '#FAEDCA',
        'borderRadius': '0 0 10px 10px'
        })
    ], style={
        'borderRadius': '10px',
        'marginBottom': '40px',
        'maxWidth': '1200px',
        'margin': '0 auto',  
        'overflow': 'hidden'
    })
    
])


@app.callback(
    [Output('ingredients-list', 'children'),
     Output('display-ingredients', 'children')],
    [Input(f'{item}-btn', 'n_clicks') for item in [
    "onion", "corn", "tomatoes", "potato", "bell pepper", "carrot", "pea", "mushroom",
    "bean", "basil", "cilantro", "red pepper", "spinach", "jalapeno", "cabbage", "zucchini",
    "cucumber", "broccoli", "lettuce", "egg", "chicken", "beef", "pork", "bacon", "sausage",
    "shrimp", "turkey", "flour", "bread", "rice", "oat", "tortilla", "wheat", "apple", "lime",
    "orange", "coconut", "strawberries", "cherries", "avocado", "cranberries", "banana", "milk",
    "cheese", "yogurt", "cream"
]],
    prevent_initial_call=True
)
def update_ingredients_list(*args):
    """ Updates ingredients table based on user input"""
    triggered_id = ctx.triggered_id
    ingredient = triggered_id.split('-')[0] if ctx.triggered_id else ''
    
    # Attempt to insert the ingredient
    success = insert_ingredient(ingredient)
    
    # Fetch the updated list of ingredients to display
    ingredients = fetch_ingredients()
    list_items = [html.Li(ingredient[0]) for ingredient in ingredients]
    
    # Create feedback message
    feedback_message = f"Added: {ingredient}" if success else f"Duplicate not added: {ingredient}"
    return feedback_message, list_items

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)

## Getting Recommended Recipes Based on User Input

In [8]:
query = "SELECT * FROM ingredients"

# Read the query results into a DataFrame
df = pd.read_sql_query(query, recipes_db)
ings = [df["ingredient"][i] for i in range(len(df))]
ings

['spinach', 'cabbage', 'oat']

In [9]:
df = query_recipes("recipes_db.sqlite", 
                   ingredients=ings
                   )
df.head(3)

,recipe_name,category_name,rating,prep_time,cook_time,additional_time,total_time,num_servings_per_recipe,ingredients_list,direction_list,...,fiber (g),sugar (g),cholesterol (mg),vitamin_c (mg),calcium (mg),iron (mg),potassium (mg),recipe_link,main_ingredients,main_ing_len
820,Barbequed Cabbage,Camping Recipes,4.5,10.0,15.0,0.0,25.0,4,"cabbage, cut into thick shreds,soy sauce,butte...",Preheat an outdoor grill for medium heat.\n D...,...,4.0,5.0,31.0,54.0,70.0,1.0,327.0,https://www.allrecipes.com/recipe/146253/barbe...,[cabbage],1
7504,Slow Cooker Oatmeal,Oatmeal,3.6,2.0,480.0,0.0,482.0,4,"oats,water,salt,half-and-half cream,brown suga...","Just before going to bed, combine the oats an...",...,2.0,14.0,22.0,1.0,91.0,1.0,172.0,https://www.allrecipes.com/recipe/88691/slow-c...,[oat],1
11120,Chef John's Chocolate Granola,Granola,4.9,10.0,60.0,0.0,70.0,14,"packed brown sugar,maple syrup,vegetable oil,v...",Preheat the oven to 250 degrees F (120 degree...,...,2.0,9.0,NaN,NaN,28.0,1.0,116.0,https://www.allrecipes.com/recipe/264951/chef-...,[oat],1
